The **task** is to write a script AppData, that will receive an **ID** of the app from iTunes and add the results of the search for each word in the app name to an **sqlite3** database called **apps.db**, in the table **search**. Each row should contain the following fields:

**app id**,

**word** - the word from a search query,

**position** - the position of the app in the iTunes search results by that word,

**date** - the date when the script was executed.

In [1]:
import requests as r
import sqlite3
from datetime import date
import re

In [98]:
def AppData (id):
    url = f'https://itunes.apple.com/lookup?id={id}' #adding the id to the link
    response = r.get(url)
    if response.ok:  #checking if the response status is OK (200)
        result=response.json()
    else:
        print ("Execution failed (search by ID)")
    if 'results' in result and len(result['results']) != 0: #checking that results are not empty
        app_name=result['results'][0]['trackName'] #extracting the app name from the results
    else:
        print(f'Error, data on app ID {id} not found')
    words = re.findall(r'\w+', app_name)  #extracting words from the app name using regex
    current_date = date.today().isoformat() #getting the current date in "yyyy-mm-dd" format
    con = sqlite3.connect("apps.db") #connecting to the database
    cur = con.cursor()
    #Creating the table if it doesn't exist
    cur.execute('''
    CREATE TABLE IF NOT EXISTS search (
        app_id INTEGER,
        word TEXT,
        position INTEGER,
        date TEXT
    )
    ''')
    for word in words:
        prep_and_articles=('in','the','a','and','by','as','at','with','of','for','on','into','to')
        #the tuple containing common prepositions and articles to exclude from searches
        if word.lower() not in prep_and_articles: 
            word_url = f'https://itunes.apple.com/search?term={word}&entity=software&limit=200'#by default the limit is 50 pages, I change it to maximum 200
            response_search = r.get(word_url)
            if response_search.ok:  
                result_search=response_search.json()
            else:
                print ("Execution failed (search by words)") 
            request=word #creating the variable with a word from the app name
            position = None #initializing position variable to track app's position in search results 
            if 'results' in result_search and len(result_search['results']) != 0: 
                for index, element in enumerate (result_search['results']): 
                    if element['trackId'] == id: #Finding the position of the app comparing trackId with the provided app ID
                        position = index + 1 #since the index starts with 0
                        break
            else:
                print("Empty search result")
            obtained_data=[id, word, position, current_date]
            cur.execute("INSERT INTO search VALUES(?,?,?,?)", obtained_data) #inserting the data
            print(f'Inserted: {obtained_data}')
            if position is None:
                print(f'App not found for word: {word}') #If position was not found, I print the warning, but add None to the table
    con.commit()
    con.close()